In [1]:
%pylab inline 

Populating the interactive namespace from numpy and matplotlib


In [2]:
%load_ext autoreload
%autoreload 2
from analysis_toolbox import *
from qualityBaseline import * 

In [3]:
save_db_as_csv(connect=False)
img_info,object_tbl,bb_info,hit_info = load_info()
#Load COCO annotations 
dataDir='../../coco/'
dataType='train2014'
annFile='%s/annotations/instances_%s.json'%(dataDir,dataType)
# initialize COCO api for instance annotations
coco=COCO(annFile)
ground_truth = pd.read_csv("../../data/object_ground_truth.csv")
worker_info = pd.read_csv("../../data/worker.csv",skipfooter=1)
my_BBG  = pd.read_csv("my_ground_truth.csv")


loading annotations into memory...
Done (t=19.21s)
creating index...
index created!


/Users/dorislee/.virtualenvs/py27dev/lib/python2.7/site-packages/pandas/io/parsers.py:648: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skip_footer; you can avoid this warning by specifying engine='python'.
  ParserWarning)


In [5]:
print obj_x_locs
print obj_y_locs

[[360, 364, 368, 379, 390, 399, 407, 406, 402, 412, 420, 426, 415, 410, 389, 375, 366, 360], [375.62, 402.51999999999998, 361.32999999999998, 406.72000000000003, 360.49000000000001, 367.22000000000003, 378.98000000000002, 393.26999999999998, 420.17000000000002, 416.80000000000001, 419.32999999999998, 411.75999999999999, 394.11000000000001, 371.42000000000002, 367.22000000000003, 405.04000000000002]]
[[162, 156, 137, 125, 123, 124, 137, 146, 159, 163, 177, 191, 206, 192, 192, 188, 182, 162], [186.38, 130.06999999999999, 172.93000000000001, 157.81, 162.84999999999999, 186.38, 121.67, 185.53999999999999, 193.94999999999999, 194.78999999999999, 180.5, 167.88999999999999, 123.34999999999999, 133.43000000000001, 149.40000000000001, 146.03999999999999]]


In [41]:
x,y = zip(*set(zip(coco_x_locs,coco_y_locs)))

In [45]:
print list(x)
print type(y )

[19.420000000000002, 39.909999999999997, 15.1, 50.700000000000003, 15.1, 30.199999999999999, 0.0, 2.1600000000000001, 37.75]
<type 'tuple'>


In [ ]:
in the COCO data there are duplicate points in the BB

In [4]:
for i in np.arange(len(img_info)):
    img_name = img_info["filename"][i]
    if 'COCO' in img_name:
        img_id = int(img_name.split('_')[-1])
        filtered_object_tbl = object_tbl[object_tbl["image_id"]==i+1]
        annIds = coco.getAnnIds(imgIds=img_id, iscrowd=None)
        anns = coco.loadAnns(annIds)
        #for oid,bbx_path,bby_path in zip(bb_info["object_id"],bb_info["x_locs"],bb_info["y_locs"]):
        for bb in bb_info.iterrows():
            oid = bb[1]["object_id"]
            bbx_path= bb[1]["x_locs"]
            bby_path= bb[1]["y_locs"]
            if int(object_tbl[object_tbl.object_id==oid].image_id) ==i+1:
                worker_x_locs,worker_y_locs= process_raw_locs([bbx_path,bby_path])
                ground_truth_match = ground_truth[ground_truth.id==str(oid)]
                COCO_id = int(ground_truth_match["COCO_annIds"])

                #COCO-Annotations
                for ann in anns:
                    if COCO_id==-1:
                        #No BB for this object collected by MSCOCO
                        pass
                    elif ann['id'] == COCO_id: 
#                         print COCO_id
                        for annBB in ann["segmentation"]:
                            coco_x_locs,coco_y_locs = process_raw_locs(annBB,COCO=True)
                            coco_x_locs,coco_y_locs = zip(*set(zip(coco_x_locs,coco_y_locs)))
                            obj_x_locs = [worker_x_locs,list(coco_x_locs)]
                            obj_y_locs = [worker_y_locs,list(coco_y_locs)]
                            bb_info = bb_info.set_value(bb[0],"Jaccard [COCO]",majority_vote(obj_x_locs,obj_y_locs))
                            bb_info = bb_info.set_value(bb[0],"Precision [COCO]",precision(obj_x_locs,obj_y_locs))
                            bb_info = bb_info.set_value(bb[0],"Recall [COCO]",recall(obj_x_locs,obj_y_locs))    
                            print "here COCO"
                            bb_info = bb_info.set_value(bb[0],"Munkres Euclidean [COCO]",MunkresEuclidean(obj_x_locs,obj_y_locs))
                my_ground_truth_match = my_BBG[my_BBG.object_id==oid]
                my_x_locs,my_y_locs =  process_raw_locs([my_ground_truth_match["x_locs"].iloc[0],my_ground_truth_match["y_locs"].iloc[0]])
                obj_x_locs = [worker_x_locs,my_x_locs]
                obj_y_locs = [worker_y_locs,my_y_locs]
                bb_info = bb_info.set_value(bb[0],"Jaccard [Self]",majority_vote(obj_x_locs,obj_y_locs))   
                bb_info = bb_info.set_value(bb[0],"Precision [Self]",precision(obj_x_locs,obj_y_locs))
                bb_info = bb_info.set_value(bb[0],"Recall [Self]",recall(obj_x_locs,obj_y_locs))
                print "here"
                bb_info = bb_info.set_value(bb[0],"Munkres Euclidean [Self]",MunkresEuclidean(obj_x_locs,obj_y_locs))
                #bb_info = bb_info.set_value(bb[0],"Munkres Euclidean [Self]",DistAllWorkers(obj_x_locs,obj_y_locs))
# replace all NAN values with -1, these are entries for which we don't have COCO ground truth
bb_info = bb_info.fillna(-1)

here
here
here
here
(19.420000000000002, 39.909999999999997, 15.1, 50.700000000000003, 15.1, 30.199999999999999, 0.0, 2.1600000000000001, 37.75)
here COCO
<type 'list'>
<type 'list'>
(413.12, 368.89999999999998, 292.31, 347.32999999999998, 423.91000000000003, 407.73000000000002, 331.14999999999998, 180.13, 398.01999999999998, 373.20999999999998, 269.66000000000003, 303.10000000000002, 298.79000000000002, 369.98000000000002, 363.50999999999999, 257.80000000000001, 354.88, 294.47000000000003, 368.89999999999998, 303.10000000000002, 300.94, 328.99000000000001, 189.84)
here COCO
<type 'list'>
<type 'list'>
here
(509.13, 508.23000000000002, 505.66000000000003, 509.72000000000003, 510.35000000000002, 511.11000000000001, 496.93000000000001, 511.31999999999999, 506.63999999999999, 501.55000000000001, 509.0, 497.00999999999999, 496.02999999999997, 496.58999999999997, 502.19, 498.20999999999998, 511.47000000000003, 510.95999999999998, 495.61000000000001, 511.70999999999998, 502.06999999999999, 5

KeyboardInterrupt: 

In [9]:
bb_info[~bb_info["Munkres Euclidean [COCO]"].isnull()]

,object_id,worker_id,x_locs,y_locs,Jaccard [Self],Precision [Self],Recall [Self],Munkres Euclidean [Self],Jaccard [COCO],Precision [COCO],Recall [COCO],Munkres Euclidean [COCO]
31,1,3,"[316, 300, 295, 304, 316, 331, 363, 366, 354, ...","[148, 183, 207, 228, 240, 244, 218, 205, 154, ...",1.000000,1.000000,1.000000,0.00000,0.050654,0.191872,0.079944,3977.968826
32,6,3,"[502, 497, 497, 506, 512, 512, 512, 509, 508]","[203, 215, 251, 251, 242, 233, 216, 209, 200]",1.000000,1.000000,1.000000,0.00000,0.225392,0.225680,1.585378,288.998149
51,7,3,"[361, 368, 368, 380, 402, 405, 405, 404, 410, ...","[166, 161, 148, 126, 122, 134, 146, 162, 169, ...",1.000000,1.000000,1.000000,0.00000,0.032403,0.032465,0.093411,1023.703426
145,7,71,"[406, 407, 405, 397, 384, 374, 367, 368, 361, ...","[142, 157, 128, 124, 122, 131, 146, 157, 161, ...",0.537109,0.909643,0.567406,858.65755,0.052637,0.052763,0.094699,552.339481


the ordering is messed up so the jaccard and other mesure got screwed up, we need to remove duplicate in a list but preserve the ordering of the list (set is unordered) 

we should also just do this inside the MunkresEuclidean function and not inside the big function because the other functions are able to handle duplicate ok

In [12]:
from collections import OrderedDict

In [16]:
list(OrderedDict.fromkeys(zip(worker_x_locs,worker_y_locs)))

[(360, 162),
 (364, 156),
 (368, 137),
 (379, 125),
 (390, 123),
 (399, 124),
 (407, 137),
 (406, 146),
 (402, 159),
 (412, 163),
 (420, 177),
 (426, 191),
 (415, 206),
 (410, 192),
 (389, 192),
 (375, 188),
 (366, 182)]

In [23]:
coco_x_locs,coco_y_locs = zip(*list(OrderedDict.fromkeys(zip(coco_x_locs,coco_y_locs))))

In [24]:
coco_x_locs

(375.62,
 402.51999999999998,
 361.32999999999998,
 406.72000000000003,
 360.49000000000001,
 367.22000000000003,
 378.98000000000002,
 393.26999999999998,
 420.17000000000002,
 416.80000000000001,
 419.32999999999998,
 411.75999999999999,
 394.11000000000001,
 371.42000000000002,
 367.22000000000003,
 405.04000000000002)

In [ ]:
zip(coco_x_locs,coco_y_locs)
obj_x_locs = [worker_x_locs,list(coco_x_locs)]
obj_y_locs = [worker_y_locs,list(coco_y_locs)]


In [ ]:

def MunkresEuclidean(obj_x_locs,obj_y_locs,numPts=50,PRINT=False):
    '''
    Given two worker's responses, 
    Compares Euclidean distances of all points in the polygon, 
    then find the best matching (min dist) config via Kuhn-Munkres
    '''
    if obj_x_locs[0]==obj_x_locs[1] and obj_y_locs[0]==obj_y_locs[1] :
        return 0
    print type(obj_x_locs[1])
    print type(obj_y_locs[1])
    
    interpolated_obj_x_locs,interpolated_obj_y_locs = parametric_interpolate(obj_x_locs,obj_y_locs,numPts=numPts,PLOT=PRINT)
    
    polygon1 = zip(interpolated_obj_x_locs[0],interpolated_obj_y_locs[0])
    polygon2 = zip(interpolated_obj_x_locs[1],interpolated_obj_y_locs[1])

    matrix = spatial.distance.cdist(polygon1,polygon2,'euclidean')
    
    m = Munkres()
    try:
        indexes = m.compute(np.ma.masked_equal(matrix,0))
        total = 0
        for row, column in indexes:
            value = matrix[row][column]
            total += value
            if PRINT: print '(%d, %d) -> %.2f' % (row, column, value)
        return total         
    except(ValueError):
        print "bad"
        return 0

In [22]:
list(process_raw_locs([bbx_path,bby_path])

[([432,
   437,
   443,
   456,
   454,
   474,
   483,
   483,
   486,
   480,
   480,
   495,
   501,
   504,
   499,
   482,
   466,
   465,
   453,
   431,
   432],),
 ([208,
   195,
   173,
   168,
   136,
   130,
   136,
   142,
   151,
   159,
   165,
   174,
   186,
   203,
   211,
   208,
   211,
   222,
   226,
   216,
   208],)]

In [37]:
img_info,object_tbl,bb_info,hit_info = load_info()

In [38]:
for i in np.arange(len(img_info)):
    img_name = img_info["filename"][i]
    if 'COCO' in img_name:
        img_id = int(img_name.split('_')[-1])
        filtered_object_tbl = object_tbl[object_tbl["image_id"]==i+1]
        annIds = coco.getAnnIds(imgIds=img_id, iscrowd=None)
        anns = coco.loadAnns(annIds)
        #for oid,bbx_path,bby_path in zip(bb_info["object_id"],bb_info["x_locs"],bb_info["y_locs"]):
        for bb in bb_info.iterrows():
            oid = bb[1]["object_id"]
            bbx_path= bb[1]["x_locs"]
            bby_path= bb[1]["y_locs"]
            if int(object_tbl[object_tbl.object_id==oid].image_id) ==i+1:
                worker_x_locs,worker_y_locs= process_raw_locs([bbx_path,bby_path])
                worker_x_locs,worker_y_locs = zip(*list(OrderedDict.fromkeys(zip(worker_x_locs,worker_y_locs))))
                ground_truth_match = ground_truth[ground_truth.id==str(oid)]
                COCO_id = int(ground_truth_match["COCO_annIds"])

                #COCO-Annotations
                for ann in anns:
                    if COCO_id==-1:
                        #No BB for this object collected by MSCOCO
                        pass
                    elif ann['id'] == COCO_id: 
#                         print COCO_id
                        for annBB in ann["segmentation"]:
                            coco_x_locs,coco_y_locs = process_raw_locs(annBB,COCO=True)
                            #Remove duplicates
                            coco_x_locs,coco_y_locs = zip(*list(OrderedDict.fromkeys(zip(coco_x_locs,coco_y_locs))))
                            obj_x_locs = [list(worker_x_locs),list(coco_x_locs)]
                            obj_y_locs = [list(worker_y_locs),list(coco_y_locs)]
                            bb_info = bb_info.set_value(bb[0],"Jaccard [COCO]",majority_vote(obj_x_locs,obj_y_locs))
                            bb_info = bb_info.set_value(bb[0],"Precision [COCO]",precision(obj_x_locs,obj_y_locs))
                            bb_info = bb_info.set_value(bb[0],"Recall [COCO]",recall(obj_x_locs,obj_y_locs))    
                            print "here COCO"
                            bb_info = bb_info.set_value(bb[0],"Munkres Euclidean [COCO]",MunkresEuclidean(obj_x_locs,obj_y_locs))
                my_ground_truth_match = my_BBG[my_BBG.object_id==oid]
                my_x_locs,my_y_locs =  process_raw_locs([my_ground_truth_match["x_locs"].iloc[0],my_ground_truth_match["y_locs"].iloc[0]])
                my_x_locs,my_y_locs = zip(*list(OrderedDict.fromkeys(zip(my_x_locs,my_y_locs))))
                obj_x_locs = [list(worker_x_locs),list(my_x_locs)]
                obj_y_locs = [list(worker_y_locs),list(my_y_locs)]
                bb_info = bb_info.set_value(bb[0],"Jaccard [Self]",majority_vote(obj_x_locs,obj_y_locs))   
                bb_info = bb_info.set_value(bb[0],"Precision [Self]",precision(obj_x_locs,obj_y_locs))
                bb_info = bb_info.set_value(bb[0],"Recall [Self]",recall(obj_x_locs,obj_y_locs))
                print "here"
                bb_info = bb_info.set_value(bb[0],"Munkres Euclidean [Self]",MunkresEuclidean(obj_x_locs,obj_y_locs))
                #bb_info = bb_info.set_value(bb[0],"Munkres Euclidean [Self]",DistAllWorkers(obj_x_locs,obj_y_locs))
# replace all NAN values with -1, these are entries for which we don't have COCO ground truth
bb_info = bb_info.fillna(-1)

here
here
here
here
here COCO
<type 'list'>
<type 'list'>
here COCO
<type 'list'>
<type 'list'>
here
here COCO
<type 'list'>
<type 'list'>
here
here COCO
<type 'list'>
<type 'list'>
here
here COCO
<type 'list'>
<type 'list'>
here
<type 'list'>
<type 'list'>
here COCO
<type 'list'>
<type 'list'>
here
<type 'list'>
<type 'list'>
here
<type 'list'>
<type 'list'>
here
<type 'list'>
<type 'list'>
here COCO
<type 'list'>
<type 'list'>
bad
here COCO
<type 'list'>
<type 'list'>
bad
here
<type 'list'>
<type 'list'>
bad
here
<type 'list'>
<type 'list'>
here COCO
<type 'list'>
<type 'list'>
here
<type 'list'>
<type 'list'>
here
<type 'list'>
<type 'list'>
here COCO
<type 'list'>
<type 'list'>
here
<type 'list'>
<type 'list'>
here COCO
<type 'list'>
<type 'list'>
here
<type 'list'>
<type 'list'>
here COCO
<type 'list'>
<type 'list'>
bad
here COCO
<type 'list'>
<type 'list'>
bad
here
<type 'list'>
<type 'list'>
bad
here
<type 'list'>
<type 'list'>
here
<type 'list'>
<type 'list'>
here
<type 'list'>

KeyboardInterrupt: 

In [39]:
bb_info[~bb_info["Munkres Euclidean [Self]"].isnull()]

,object_id,worker_id,x_locs,y_locs,Jaccard [Self],Precision [Self],Recall [Self],Munkres Euclidean [Self],Jaccard [COCO],Precision [COCO],Recall [COCO],Munkres Euclidean [COCO]
5,5,3,"[432, 437, 443, 456, 454, 474, 483, 483, 486, ...","[208, 195, 173, 168, 136, 130, 136, 142, 151, ...",1.000000,1.000000,1.000000,0.000000,NaN,NaN,NaN,NaN
7,4,3,"[91, 96, 93, 88, 85, 88, 108, 124, 134, 134, 1...","[335, 355, 381, 418, 444, 455, 462, 452, 442, ...",1.000000,1.000000,1.000000,0.000000,NaN,NaN,NaN,NaN
9,3,3,"[47, 90, 97, 90, 86, 47, 52]","[319, 340, 360, 409, 422, 407, 340]",1.000000,1.000000,1.000000,0.000000,NaN,NaN,NaN,NaN
30,2,3,"[187, 210, 242, 251, 227, 187]","[336, 317, 337, 415, 431, 415]",1.000000,1.000000,1.000000,0.000000,NaN,NaN,NaN,NaN
31,1,3,"[316, 300, 295, 304, 316, 331, 363, 366, 354, ...","[148, 183, 207, 228, 240, 244, 218, 205, 154, ...",1.000000,1.000000,1.000000,0.000000,0.171251,0.962922,0.172388,4880.491393
32,6,3,"[502, 497, 497, 506, 512, 512, 512, 509, 508]","[203, 215, 251, 251, 242, 233, 216, 209, 200]",1.000000,1.000000,1.000000,0.000000,0.866481,0.906596,0.951415,149.827434
51,7,3,"[361, 368, 368, 380, 402, 405, 405, 404, 410, ...","[166, 161, 148, 126, 122, 134, 146, 162, 169, ...",1.000000,1.000000,1.000000,0.000000,0.547902,0.576956,0.915826,769.711834
145,7,71,"[406, 407, 405, 397, 384, 374, 367, 368, 361, ...","[142, 157, 128, 124, 122, 131, 146, 157, 161, ...",0.537109,0.909643,0.567406,864.338040,0.860130,0.929448,0.920278,342.613347
156,7,92,"[360, 364, 368, 379, 390, 399, 407, 406, 402, ...","[162, 156, 137, 125, 123, 124, 137, 146, 159, ...",0.595807,0.905263,0.635427,621.122617,0.837848,0.865046,0.963831,304.741757
163,3,99,"[51, 49, 49, 88, 96]","[320, 370, 407, 419, 342]",0.903386,0.964481,0.934476,576.368518,NaN,NaN,NaN,NaN
